# Why Julia?

#### _"We want a language that’s open source, with a liberal license. We want the speed of C with the dynamism of Ruby. We want a language that’s homoiconic, with true macros like Lisp, but with obvious, familiar mathematical notation like Matlab. We want something as usable for general programming as Python, as easy for statistics as R, as natural for string processing as Perl, as powerful for linear algebra as Matlab, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled._

#### _(Did we mention it should be as fast as C?)"_

Basically they want the moon on a stick. They're pretty close to getting it.

## This `tutorial'

This is a whirlwind tour of Julia. We'll cover:
- Nerdy things which are neat but minor.
- Matrix operations.
- Functions, multiple dispatch, introspection.
- Some other stuff.
- Parallelisation.
- Interoperability.

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

## Incomplete discussion of the basics

<img src="imgs/billy.jpg" width="500", align="left">

#### Numeric types

Default numeric type is double precision: Float64. Also supports, Float32, Int64, etc

In [73]:
example = (one(Float32), rand(Float64), zero(Int64))

(1.0f0,0.3417192788727441,0)

Division causes type upgrade. This allows type stability in programs.

In [74]:
1/3, typeof(1/3)

(0.3333333333333333,Float64)

#### Control flow

In [142]:
v = rand(5)
for i in 1:length(v)
    if i > 2
        v[i] = sqrt(i)
    elseif i==1
        v[i] = -1
    else
        v[i] = 0
    end
end
println(v)

[-1.0,0.0,1.73205,2.0,2.23607]


#### Data types and functions

There are no classes. 
There are data types, and functions which act on data types.

In [156]:
type Foo
    bar::Float64
    baz::Array{Int64,1}
end

a = Foo(3.7, [2,3])
b = Foo(2.5, [1,5])

Foo(2.5,[1,5])

In [157]:
import Base: *
function *(a::Foo, b::Foo)
    return Foo(a.bar*b.bar, a.baz.*b.baz)
end

* (generic function with 182 methods)

In [158]:
a*b

Foo(9.25,[2,15])

__________________________________________________________________________________________________________________

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

# Fun things, tres nerd.

<img src="imgs/220px-Pocketp.gif" width="150", align="left">

### Rational numbers

In [76]:
r = 3//7 * 7//5

3//5

In [77]:
typeof(r)

Rational{Int64}

In [78]:
R = 1.//collect(1:5)

5-element Array{Rational{Int64},1}:
 1//1
 1//2
 1//3
 1//4
 1//5

Let's factorise a matrix. $\mathbf{M = LU}$

In [79]:
M= R*R'

5×5 Array{Rational{Int64},2}:
 1//1  1//2   1//3   1//4   1//5 
 1//2  1//4   1//6   1//8   1//10
 1//3  1//6   1//9   1//12  1//15
 1//4  1//8   1//12  1//16  1//20
 1//5  1//10  1//15  1//20  1//25

In [80]:
lufactM = lu(M)

(
Rational{Int64}[1//1 0//1 … 0//1 0//1; 1//2 1//1 … 0//1 0//1; … ; 1//4 0//1 … 1//1 0//1; 1//5 0//1 … 0//1 1//1],

Rational{Int64}[1//1 1//2 … 1//4 1//5; 0//1 0//1 … 0//1 0//1; … ; 0//1 0//1 … 0//1 0//1; 0//1 0//1 … 0//1 0//1],

[1,2,3,4,5])

In [81]:
lufactM[2]

5×5 Array{Rational{Int64},2}:
 1//1  1//2  1//3  1//4  1//5
 0//1  0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1  0//1
 0//1  0//1  0//1  0//1  0//1

__________________________________________________________________________________________________________________

### High precision data types

Let's try and evaluate a really big number $200!=200 \times199 \times 198...$


In [82]:
factorial(200) #woops - this is a big number

LoadError: LoadError: OverflowError()
while loading In[82], in expression starting on line 1

In [83]:
n = 200*one(BigInt)

200

In [89]:
typeof(n)

BigInt

In [90]:
bigfact = factorial(n)

788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000

Suppose we have a small $x = \frac{1}{10^{500}}$. Now obvs $\frac{x}{x}=1$.

In [87]:
x_Float64 = 1.0 / (10.0^-500)  #woops - divide by 'zero'
x_Float64/x_Float64

NaN

In [88]:
x_BigFloat = 1.0 / ((10one(BigFloat))^-500)
x_BigFloat/x_BigFloat

1.000000000000000000000000000000000000000000000000000000000000000000000000000000

Seems silly, but this stuff can be super annoying in numerical programming. There's a speed penalty, but maybe you want to optimise your code _after_ you get it working. And/or assure yourself the idea itself isn't rubbish, which it probably is, if you are me.

### Unicode

In [30]:
π

π = 3.1415926535897...

In [122]:
ϕ = (1 + sqrt(5))/2

1.618033988749895

In [125]:
🕢 = now()

2016-11-10T17:38:48.166

__________________________________________________________________________________________________________________

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

# Wow computing, much science

<img src="imgs/c_new_york.png" width="1000" align="left">

## Linear algebra

Matrix operations are default: 
- multiplication *
- arithmetic +, -
- solve \

Julia is column major (M[:,1] is quicker than M[1,:])

In [315]:
A = randn(5,5);
B = A' * A

5×5 Array{Float64,2}:
  7.44719    2.32404   -0.515429  -3.83385    0.581364
  2.32404   13.9877    -0.245746  -1.23764   -3.19265 
 -0.515429  -0.245746   3.44641   -3.69492   -1.0904  
 -3.83385   -1.23764   -3.69492    9.02937    0.525965
  0.581364  -3.19265   -1.0904     0.525965   1.33075 

For elementwise operations, add a . before the operator.

In [316]:
H = A .* A

5×5 Array{Float64,2}:
 1.31509   0.36906  0.15812   0.734707  0.410736
 0.30145   1.11984  2.32395   0.871804  0.103828
 4.55906   3.23344  0.478224  4.93482   0.15735 
 0.981026  5.85091  0.153583  0.358542  0.522184
 0.290561  3.41446  0.332528  2.12949   0.136651

Comes packed with blas and suitesparse out of the box.

In [420]:
C = chol(B)

5×5 UpperTriangular{Float64,Array{Float64,2}}:
 2.72895  0.851624  -0.188874   -1.40488     0.213035
  ⋅       3.64176   -0.0233118  -0.0113168  -0.926494
  ⋅        ⋅         1.84667    -2.14469    -0.580374
  ⋅        ⋅          ⋅          1.56712    -0.274357
  ⋅        ⋅          ⋅           ⋅          0.121934

Solving a system of equations $ \mathbf{Bx = y} $

In [431]:
y  = randn(5)
C \ y

5-element Array{Float64,1}:
 -0.0374569
  0.495167 
  0.921966 
  0.132369 
  1.89002  

### Sparse matrices

The point here is that Julia treats sparse matrices properly out of the box.

In [319]:
S = sprand(100,100,0.1);
S[1:2,:]

2×100 sparse matrix with 23 Float64 nonzero entries:
	[1  ,   5]  =  0.599002
	[2  ,   5]  =  0.955839
	[2  ,  32]  =  0.93147
	[1  ,  36]  =  0.211836
	[1  ,  39]  =  0.110356
	[1  ,  45]  =  0.559659
	[1  ,  50]  =  0.960968
	[2  ,  52]  =  0.533508
	[1  ,  56]  =  0.88769
	[2  ,  59]  =  0.575355
	⋮
	[1  ,  80]  =  0.706075
	[2  ,  80]  =  0.732738
	[1  ,  87]  =  0.117349
	[1  ,  88]  =  0.712092
	[2  ,  88]  =  0.683166
	[2  ,  91]  =  0.151015
	[1  ,  94]  =  0.815216
	[2  ,  95]  =  0.775434
	[2  ,  96]  =  0.823778
	[1  ,  97]  =  0.25485
	[2  ,  98]  =  0.388228

In [320]:
R = S'*S;

In [321]:
cholR = cholfact(R)

Base.SparseArrays.CHOLMOD.Factor{Float64}
type:          LLt
method: supernodal
maxnnz:          0
nnz:          4784


In [322]:
v = randn(size(R)[1]);

In [323]:
@elapsed cholR\v

5.9378e-5

In [114]:
@elapsed R\v

1.156225688

## Probability

In [160]:
using Distributions

In [162]:
F = Distributions.Laplace(0,1)

Distributions.Laplace{Float64}(μ=0.0, θ=1.0)

In [164]:
mean(F)

0.0

In [166]:
var(F)

2.0

In [169]:
rand(F,5)

5-element Array{Float64,1}:
  0.192603
  2.28452 
 -1.68913 
 -0.120368
 -0.399439

In [170]:
cdf(F, 4)

0.9908421805556329

---------------------------------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

# Functions

<img src="imgs/trump_wall.jpg" width="500" align="left">

### Multiple dispatch

In [126]:
function dostuff(N)
    v = zeros(N)
    for i in 1:N
        v[i] = i^2
    end
    return v
end

dostuff (generic function with 2 methods)

In [34]:
dostuff(5)

5-element Array{Float64,1}:
  1.0
  4.0
  9.0
 16.0
 25.0

In [35]:
dostuff(π)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Irrational{:π} to an object of type Array{Float64,N}
This may have arisen from a call to the constructor Array{Float64,N}(...),
since type constructors fall back to convert methods.
while loading In[35], in expression starting on line 1

In [46]:
function dostuff(s::String)
    return s^2
end

dostuff (generic function with 2 methods)

In [47]:
dostuff("hello")

"hellohello"

### Introspection

Introspect on your code with macros, including *inter alia* :
 - @code_warntype what types are getting used in your function, great for performance tweaking.
 - @code_native the machine code in case you don't believe it's actually a compiled language...

In [12]:
@code_warntype dostuff(3)

Variables:
  #self#::#dostuff
  N::Int64
  v::Array{Float64,1}
  #temp#::Int64
  i::Int64

Body:
  begin 
      v::Array{Float64,1} = $(Expr(:invoke, LambdaInfo for fill!(::Array{Float64,1}, ::Float64), :(Base.fill!), :((Core.ccall)(:jl_alloc_array_1d,(Core.apply_type)(Core.Array,Float64,1)::Type{Array{Float64,1}},(Core.svec)(Core.Any,Core.Int)::SimpleVector,Array{Float64,1},0,N,0)::Array{Float64,1}), :((Base.box)(Float64,(Base.sitofp)(Float64,0))))) # line 3:
      SSAValue(4) = (Base.select_value)((Base.sle_int)(1,N::Int64)::Bool,N::Int64,(Base.box)(Int64,(Base.sub_int)(1,1)))::Int64
      #temp#::Int64 = 1
      5: 
      unless (Base.box)(Base.Bool,(Base.not_int)((#temp#::Int64 === (Base.box)(Int64,(Base.add_int)(SSAValue(4),1)))::Bool)) goto 16
      SSAValue(5) = #temp#::Int64
      SSAValue(6) = (Base.box)(Int64,(Base.add_int)(#temp#::Int64,1))
      i::Int64 = SSAValue(5)
      #temp#::Int64 = SSAValue(6) # line 4:
      SSAValue(2) = (Base.Math.box)(Base.Math.Float64,(Base.Mat

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

## Metaprogramming

<img src="imgs/dwarf_fortress.jpg" width="800" align="left">

Code is data, and can be operated on and changed by other code. 


Before they get executed, your instructions are parsed. We can hang on to the parsed-but-not-yet-executed instructions with `Expr` objects.

In [50]:
M = randn(5,5)
somecode = :(M*M)

:(M * M)

In [51]:
typeof(somecode)

Expr

We can evaluate the instruction at a later point.

In [52]:
eval(somecode)

5×5 Array{Float64,2}:
 -5.05223   -2.04804    2.00255    2.05285   -3.09112 
 -0.210684  -2.17474    0.138667  -0.521553  -3.72601 
 -1.64465    2.01863    3.12356    2.23276    1.79974 
  2.59097    0.148061  -0.175742  -0.182486   0.280778
  7.53026   -1.37359   -2.57355   -3.69354   -2.64266 

In [53]:
M = (1//13)M 

5×5 Array{Float64,2}:
  0.0545127  -0.0970267   -0.0622521  -0.0542917  -0.124665 
  0.123862   -0.00875638  -0.0478131  -0.0731106  -0.0415542
 -0.0980772  -0.0426667   -0.0858431  -0.0579495  -0.024355 
  0.0476063  -0.00868794  -0.0409868   0.0225544   0.0446009
  0.195479    0.0866865   -0.0243426  -0.0451609   0.117286 

In [54]:
eval(somecode)

5×5 Array{Float64,2}:
 -0.0298948   -0.0121186     0.0118494     0.012147    -0.0182906 
 -0.00124665  -0.0128683     0.000820513  -0.00308611  -0.0220474 
 -0.00973168   0.0119446     0.0184826     0.0132116    0.0106494 
  0.0153312    0.000876103  -0.00103989   -0.0010798    0.00166141
  0.0445577   -0.00812774   -0.0152281    -0.0218553   -0.015637  

We can even alter the behaviour of not-yet-executed code with `macro`s.

In [55]:
fieldnames(typeof(somecode))

3-element Array{Symbol,1}:
 :head
 :args
 :typ 

In [56]:
somecode.args

3-element Array{Any,1}:
 :*
 :M
 :M

For example, we could switch out matrix multiplication for elementwise multiplication.

In [57]:
macro corruptmatmult(arg)
   if arg.args[1] == :*
       arg.args[1] = :.*
   end
   return arg
end

@corruptmatmult (macro with 1 method)

In [58]:
M*M

5×5 Array{Float64,2}:
 -0.0298948   -0.0121186     0.0118494     0.012147    -0.0182906 
 -0.00124665  -0.0128683     0.000820513  -0.00308611  -0.0220474 
 -0.00973168   0.0119446     0.0184826     0.0132116    0.0106494 
  0.0153312    0.000876103  -0.00103989   -0.0010798    0.00166141
  0.0445577   -0.00812774   -0.0152281    -0.0218553   -0.015637  

In [59]:
@corruptmatmult M*M

5×5 Array{Float64,2}:
 0.00297164  0.00941418  0.00387532   0.00294759   0.0155413  
 0.0153419   7.66741e-5  0.00228609   0.00534515   0.00172675 
 0.00961913  0.00182045  0.00736904   0.00335815   0.000593168
 0.00226636  7.54804e-5  0.00167992   0.000508702  0.00198924 
 0.0382122   0.00751454  0.000592562  0.00203951   0.0137559  

------------------------------------------------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

# Parallel 

<img src="imgs/spam.jpg" width="500" align="left">

Were you just thinking, "Sure, but why would I be interested in metaprogramming?" Fret no longer. 

Let's grab some more processors.

In [98]:
rmprocs(workers()); pids = addprocs(4);

In [99]:
workers()'

1×4 Array{Int64,2}:
 2  3  4  5

In [100]:
@fetchfrom pids[2] myid()

3

In [101]:
[@fetchfrom pid randn() for pid in pids]'

1×4 Array{Float64,2}:
 -1.09215  0.0949475  0.0771  -0.121237

In [102]:
@everywhere n = myid()

In [103]:
(myid(), pids[2])

(1,3)

In [104]:
@fetchfrom pids[2] n

1

Huh? We want to retrieve the value we previously assigned to the symbol `:n` on the second processor.

In [105]:
@fetchfrom pids[2] eval(:n)

3

So we need metaprogramming to do parallel computing conveniently.

I made a small package, `ClusterUtils`, to facilitate this.

In [106]:
using ClusterUtils;
pids = workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [107]:
results = reap(pids, :(myid()*[1,2,3]))

Dict{Int64,Any} with 4 entries:
  4 => [4,8,12]
  2 => [2,4,6]
  3 => [3,6,9]
  5 => [5,10,15]

In [108]:
reduce(hcat, values(results))

3×4 Array{Int64,2}:
  4  2  3   5
  8  4  6  10
 12  6  9  15

In [109]:
reap(pids, :(x = randn(3)));

In [112]:
results = reap(pids, :(pi*x))

Dict{Int64,Any} with 4 entries:
  4 => [-2.7741,3.77419,0.737161]
  2 => [3.69268,8.17016,1.42892]
  3 => [8.813,5.06108,-5.54328]
  5 => [-1.4531,3.56357,-0.0199525]

In [113]:
reduce(vcat, values(results))

12-element Array{Float64,1}:
 -2.7741   
  3.77419  
  0.737161 
  3.69268  
  8.17016  
  1.42892  
  8.813    
  5.06108  
 -5.54328  
 -1.4531   
  3.56357  
 -0.0199525

--------------------------------------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

-----------------------------------

## Interoperability


<img src="imgs/allyour.png" width="400" align="left">

### shell

In [376]:
println(readstring(`cat /proc/meminfo`)[1:200] * "...")

MemTotal:       16389716 kB
MemFree:         7230676 kB
Buffers:          389688 kB
Cached:          3303624 kB
SwapCached:            0 kB
Active:          6160128 kB
Inactive:        1816868 kB
Acti...


In [383]:
;git log | head

commit 77100da6673305ed6d4aa66ab1c96f1c2eb4742b
Author: Matthew Pearce <mcp50@cam.ac.uk>
Date:   Mon Nov 7 16:19:12 2016 +0000

    consolidating.

commit b9cfbb54de20950f6e9a001b280bed0c97bc9aee
Author: Matthew Pearce <mcp50@cam.ac.uk>
Date:   Fri Nov 4 13:37:23 2016 +0000



### python

We can use python modules from Julia.

In [351]:
using PyCall
@pyimport sklearn.svm as svm

In [353]:
model = svm.SVC()

PyObject SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [358]:
X = randn(5,5)
y = randn(5)
z = randn(5)

5-element Array{Float64,1}:
 -0.530013 
 -0.0234911
 -0.524552 
  0.465881 
  0.639702 

In [357]:
model[:fit](X,y)

PyObject SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [362]:
model[:predict](z)

1-element Array{Float64,1}:
 -0.113438

## C

We can use C and Fortran libraries from Julia.

In [19]:
ccall((:clock, "libc"), Int32, ())

35950000

So we can wrap our 'favourite' functions:

In [20]:
gsl_acosh = (x::Float64)->ccall((:gsl_acosh, "libgsl"), Float64, (Float64, ), x)

(::#1) (generic function with 1 method)

In [21]:
gsl_acosh(2.0)

1.3169578969248166

## R

In [23]:
using RCall

In [26]:
x=randn(50,10); 
y=randn(50);

In [29]:
R"summary(lm($y~$x))"

RCall.RObject{RCall.VecSxp}

Call:
lm(formula = `#JL`$y ~ `#JL`$x)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.13088 -0.66118 -0.00699  0.54777  2.89071 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept) -0.100372   0.175394  -0.572   0.5704  
`#JL`$x1    -0.180516   0.162951  -1.108   0.2747  
`#JL`$x2     0.092657   0.186672   0.496   0.6224  
`#JL`$x3     0.130664   0.161778   0.808   0.4242  
`#JL`$x4     0.298367   0.173052   1.724   0.0926 .
`#JL`$x5    -0.005199   0.175797  -0.030   0.9766  
`#JL`$x6    -0.161587   0.160249  -1.008   0.3195  
`#JL`$x7    -0.072381   0.208705  -0.347   0.7306  
`#JL`$x8     0.376022   0.152316   2.469   0.0180 *
`#JL`$x9     0.052986   0.158418   0.334   0.7398  
`#JL`$x10    0.238552   0.174988   1.363   0.1806  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.132 on 39 degrees of freedom
Multiple R-squared:  0.2687,	Adjusted R-squared:  0.08122 
F-stati

See also Fortran, Java, Spark, Matlab etc etc.